<a href="https://colab.research.google.com/github/Riyanchaudhary/Emailclassifier/blob/main/Copy_of_Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files
uploaded = files.upload()


Saving emails - Sheet1 (2).csv to emails - Sheet1 (2) (1).csv


In [ ]:
import pandas as pd
print(df[['Snippet']].head(20))


NameError: name 'df' is not defined

In [ ]:
import pandas as pd


df = pd.read_csv("emails - Sheet1 (2).csv")


print(df.head())

                 Date                                               From  \
0  2/21/2025 13:26:55  "'Director Student Welfare, Chennai Campus' vi...   
1  2/21/2025 13:15:16  "'Director Student Welfare, Chennai Campus' vi...   
2                 NaN                                                NaN   
3                 NaN                                                NaN   
4                 NaN                                                NaN   

                                       Subject  \
0  VIT Chennai - Health Centre - Feedback Form   
1          Fwd: Magadh Mahotsav Carnival - MMC   
2                                          NaN   
3                                          NaN   
4                                          NaN   

                                             Snippet  
0  Dear Students,\r\nWe seek your feedback about ...  
1  *Thanks and Regards,*\r\n\r\n\r\n\r\n*Vellore ...  
2                                                NaN  
3                     

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text


print(df.columns)


df['clean_text'] = df['Snippet'].astype(str).apply(preprocess_text)

#clear data
print(df[['Snippet', 'clean_text']].head())

Index(['Date', 'From', 'Subject', 'Snippet'], dtype='object')
                                             Snippet  \
0  Dear Students,\r\nWe seek your feedback about ...   
1  *Thanks and Regards,*\r\n\r\n\r\n\r\n*Vellore ...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                          clean_text  
0  dear students seek feedback health centre plea...  
1  thanks regards vellore institute technology vi...  
2                                                nan  
3                                                nan  
4                                                nan  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text']).toarray()


In [ ]:
def classify_email(text):
    text = text.lower()

    if any(word in text for word in ["exam", "midterm", "timetable", "results", "grades"]):
        return "Exam Updates"
    elif any(word in text for word in ["robotics club", "sports meet", "tech fest", "webinar", "conference"]):
        return "Club Events"
    elif any(word in text for word in ["internship", "placement", "career fair", "recruitment", "hiring"]):
        return "Internship Offers"
    elif any(word in text for word in ["hostel", "mess menu", "maintenance", "warden", "electricity"]):
        return "Hostel Updates"
    else:
        return "Other"

df['Category'] = df['Snippet'].astype(str).apply(classify_email)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_features=5000)


df['Snippet'] = df['Snippet'].fillna('')

X = vectorizer.fit_transform(df['Snippet']).toarray()
print("Shape of TF-IDF matrix:", X.shape)

Shape of TF-IDF matrix: (548, 1438)


In [ ]:
from sklearn.model_selection import train_test_split

categories = df['Category'].unique()
category_map = {category: idx for idx, category in enumerate(categories)}
df['category_encoded'] = df['Category'].map(category_map)  # Map category to a number

X_train, X_test, y_train, y_test = train_test_split(X, df['category_encoded'], test_size=0.2, random_state=42)

# Print dataset size
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")


Training set size: 438, Test set size: 110


In [ ]:
print(df['Category'].value_counts())



Category
Other                516
Club Events           10
Hostel Updates         9
Exam Updates           8
Internship Offers      5
Name: count, dtype: int64
